# Siamese Network
# Euclidean Distance and Thresholding
# Bounding Box Prediction Network

In [1]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Convert the images to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute the absolute difference between the two images
diff = cv2.absdiff(gray1, gray2)

# Apply a threshold to obtain a binary image
_, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around the contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display the images with bounding boxes
cv2.imshow('Image 1', image1)
cv2.imshow('Image 2', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    intersection_x = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
    intersection_y = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
    intersection = intersection_x * intersection_y
    union = w1 * h1 + w2 * h2 - intersection
    iou = intersection / union if union > 0 else 0
    return iou


In [5]:
def calculate_map(ground_truth, predicted_boxes, iou_threshold=0.5):
    # Sort predicted boxes by confidence score
    predicted_boxes.sort(key=lambda x: x[4], reverse=True)
    true_positives = []
    false_positives = []
    for pred_box in predicted_boxes:
        pred_box_correct = False
        for gt_box in ground_truth:
            iou = calculate_iou(pred_box[:4], gt_box)
            if iou >= iou_threshold and not pred_box_correct:
                true_positives.append(pred_box)
                pred_box_correct = True
        if not pred_box_correct:
            false_positives.append(pred_box)
    # Calculate precision and recall
    precision = len(true_positives) / (len(true_positives) + len(false_positives))
    recall = len(true_positives) / len(ground_truth)
    return precision, recall


In [6]:
# Example ground truth bounding boxes
ground_truth_boxes = [(100, 100, 50, 50), (200, 200, 30, 30)]

# Example predicted bounding boxes with confidence scores
predicted_boxes = [(90, 90, 60, 60, 0.9), (200, 200, 40, 40, 0.8), (300, 300, 20, 20, 0.7)]

# Calculate IoU between two boxes
box1 = (100, 100, 50, 50)
box2 = (90, 90, 60, 60)
iou = calculate_iou(box1, box2)
print("IoU:", iou)

# Calculate mAP
precision, recall = calculate_map(ground_truth_boxes, predicted_boxes)
print("Precision:", precision)
print("Recall:", recall)


IoU: 0.6944444444444444
Precision: 0.6666666666666666
Recall: 1.0


In [7]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Convert the images to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute the absolute difference between the two images
diff = cv2.absdiff(gray1, gray2)

# Apply a threshold to obtain a binary image
_, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Prepare ground truth bounding boxes (assuming you have them)
ground_truth_boxes = [(100, 100, 50, 50), (200, 200, 30, 30)]

# Prepare predicted bounding boxes (assuming you have them)
predicted_boxes = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    predicted_boxes.append((x, y, w, h, 1.0))  # Assuming confidence score is 1.0 for all boxes

# Calculate mAP
precision, recall = calculate_map(ground_truth_boxes, predicted_boxes)

# Print mAP
print("Precision:", precision)
print("Recall:", recall)

# Draw bounding boxes around the contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display the images with bounding boxes
cv2.imshow('Image 1', image1)
cv2.imshow('Image 2', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()


Precision: 0.0
Recall: 0.0


In [48]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np

# Load the two images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Convert images to grayscale
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSI) between the two images
(score, diff) = ssim(gray_image1, gray_image2, full=True)
diff = (diff * 255).astype("uint8")

# Threshold the difference image, find contours to localize differences
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around the differences
for contour in contours:
    (x, y, w, h) = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 0, 255), 2)

# Display the images with differences highlighted
cv2.imshow("Original Image", image1)
cv2.imshow("Modified Image", image2)
cv2.imshow("Difference", diff)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [9]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np

# Load the two images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Convert images to grayscale
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSI) between the two images
(score, diff) = ssim(gray_image1, gray_image2, full=True)
diff = (diff * 255).astype("uint8")

# Threshold the difference image, find contours to localize differences
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize evolution matrix and metrics
evolution_matrix = np.zeros(gray_image1.shape)
num_changes = 0
total_change = 0

# Draw bounding boxes around the differences and update metrics
for contour in contours:
    (x, y, w, h) = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    evolution_matrix[y:y+h, x:x+w] += 1
    num_changes += 1
    total_change += cv2.countNonZero(thresh[y:y+h, x:x+w])

# Calculate quantitative metrics
average_change = total_change / num_changes if num_changes > 0 else 0
max_change = np.max(evolution_matrix)

# Display the images with differences highlighted
cv2.imshow("Original Image", image1)
cv2.imshow("Modified Image", image2)
cv2.imshow("Difference", diff)

# Display the evolution matrix
evolution_matrix_normalized = cv2.normalize(evolution_matrix, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow("Evolution Matrix", evolution_matrix_normalized)

# Print quantitative metrics
print("Number of changes:", num_changes)
print("Total change:", total_change)
print("Average change per change region:", average_change)
print("Max change in a change region:", max_change)

cv2.waitKey(0)
cv2.destroyAllWindows()


Number of changes: 23
Total change: 18849
Average change per change region: 819.5217391304348
Max change in a change region: 2.0


In [3]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np

# Load the two images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Convert images to grayscale
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSI) between the two images
(score, diff) = ssim(gray_image1, gray_image2, full=True)
diff = (diff * 255).astype("uint8")

# Threshold the difference image, find contours to localize differences
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize evolution matrix and metrics
evolution_matrix = np.zeros(gray_image1.shape)

# Draw bounding boxes around the differences and update metrics
for i, contour in enumerate(contours):
    (x, y, w, h) = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    # Draw number in the bounding box
    cv2.putText(image1, str(i + 1), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    evolution_matrix[y:y+h, x:x+w] += 1
    num_changes += 1
    total_change += cv2.countNonZero(thresh[y:y+h, x:x+w])



# Display the images with differences highlighted
cv2.imshow("Original Image", image1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
import os

def compare_images(image1_path, image2_path, output_dir=None):
    # Load the two images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Convert images to grayscale
    gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Compute the Structural Similarity Index (SSI) between the two images
    # (score, diff) = ssim(gray_image1, gray_image2, full=True)
    diff = (diff * 255).astype("uint8")

    Threshold the difference image, find contours to localize differences
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around the differences
    for i, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image1, str(i + 1), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        

    # Save the image with differences highlighted
    if output_dir is None:
        output_dir = os.path.dirname(image1_path)
    output_path = os.path.join(output_dir, 'diff_image.png')
    cv2.imwrite(output_path, image1)

    return output_path

# Example usage
result_path = compare_images('input1.png', 'input2.png')
print(f"Differences saved in: {result_path}")


UnboundLocalError: local variable 'diff' referenced before assignment